In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [4]:
df_ventas = pd.read_csv('./data/ventas.txt', delimiter = "|", infer_datetime_format=True, parse_dates=[0])

In [6]:
# vamos a guardar las ventas que tienen valores de unidades negativas
df_ventas_negativas = (
    df_ventas[lambda df: df.unidades <= 0]
    .sort_values(['unidades',"id_pos"], ascending=True)
)

En primera instancia, buscamos compras una semana antes de dichas compras negativas en el mismo establecimiento, para verificar si se han realizado una compra con la misma cantidad de unidades. Esto nos permitiría conocer cuales posiblemente sean devoluciones y no debamos quitarlas.

In [7]:
#obtenemos los establecimientos que tienen unidades negativas: 
pos_negativos = df_ventas_negativas.id_pos.unique()

#filtramos las ventas de los establecimientos y los agrupamos por su id
df_ventas_pos_negativos = df_ventas[lambda df: df.id_pos.isin(pos_negativos)]

#por cada venta negativa, en un establecimiento, 
# deberíamos buscar si 1 semana atrás tiene una venta con la misma cantidad.
def get_one_week_before(df):
    """
    está función retorna los elementos que tienen una venta en cantidad igual 
    a la venta que tiene valores nulos. 
    toma el último valor de ellos y lo retorna
    """
    id_pos = df.id_pos.iloc[0]
    ventas_id = df_ventas_negativas[lambda d: d.id_pos == id_pos].fecha
    ventas_id["rango"] = ventas_id.apply(lambda r: pd.date_range(end=r, periods=8))
    ret = pd.DataFrame().reindex_like(df).dropna()
    for t in ventas_id.rango.iteritems():
        rango = df[df.set_index(['fecha']).index.isin(t[1])]
        units = rango.unidades
        last = units.iloc[-1]
        if last < 0: 
            try:
                ret = (
                    ret
                       .append(
                           rango[lambda df: 
                                 df.unidades == -last].iloc[-1], 
                           ignore_index=True)
                      )
            except:
                pass
    return ret
    
(
    df_ventas_pos_negativos
    .groupby('id_pos')
    .apply(lambda df: get_one_week_before(df))
)

,,fecha,id_pos,unidades,canal
id_pos,,,,,
3262,0,2012-12-20,3262.0,1.0,ALMACEN
3384,0,2012-12-06,3384.0,1.0,ALMACEN
3422,0,2012-12-21,3422.0,1.0,ALMACEN
3485,0,2012-12-09,3485.0,1.0,ALMACEN
3494,0,2012-12-03,3494.0,1.0,ALMACEN
3512,0,2012-12-15,3512.0,2.0,ALMACEN
3587,0,2012-11-16,3587.0,1.0,ALMACEN
3667,0,2013-12-15,3667.0,2.0,ALMACEN
3756,0,2013-12-12,3756.0,1.0,ALMACEN


Podemos observar que tenemos 82 ventas con posible devolución en el plazo de 7 días, sobre un total de 114, es decir 32 ventas que no tienen otra que a la cual anular en un periodo de 7 días hacía atrás.

Podríamos remover solo las 32 ventas y realizar los cálculos con las demás.

Este razonamiento es un poco complejo, ya que lo interesante de analizar realmente son las ventas y no las devoluciones. Además, en la consigna de la competencia se conoce de antemano que el dataset se encuentra incompleto, para que las predicciones no se sobreajunten.

[Volver al analisis de ventas](./00-analisis-exploratorio-ventas.ipynb)